## 有很多的特征向量

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append(r'D:\pycharm\untitled')
from KNN.simple_linear_regression import linerregression
from KNN.train_test_split import train_text_split 
from sklearn import datasets

### 下面就是多元回归函数了

#### 还需要了解一点是，(x0=1 恒成立的)y=x0 x1 x2 x3 x4 x5... (其中x是特征向量)
####  还有 θ0 θ1 θ2(是系数向量)，其中那个 θ0 就是b(与x0对应的那个)
###  所以n个特征量 最后写出来了个 (n+1)维的向量
#####  都用向量来表示，所以最后y 假设函数也用向量来表示 
##### 最后预测的结果我们 都是向量的结果来弄

# 最后的 代价函数变为   (y(i)-y(i)(预测))**2 ==>   y(i)=xb * θ(都是向量)    (y-xb* θ).T  *(y-xb* θ)
##### 最后是一个数值
##### 最后满足的答案是  θ=(xb.T * xb)^(-1) * xb.T * y

但是 上面的这个算法 复杂度很高，效率低下

# 下面就会求出更简单的方法

In [2]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 


In [3]:
bost=datasets.load_boston()
x=bost.data
y=bost.target
x=x[y<50.0]
y=y[y<50.0]

In [4]:
print(x.shape)
print(y.shape)

(490, 13)
(490,)


In [7]:
x_train,y_train ,x_test,y_test= train_text_split(x,y,0.2,seed=666)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(392, 13)
(392,)
(98, 13)
(98,)


In [8]:
from KNN.simple_linear_regression import linerregression

In [9]:
reg=linerregression()
reg.fit(x_train,y_train)
print('截距',reg.inter_,reg.inter_.shape,sep='\n')
print('除了截距的',reg.coef_,reg.coef_.shape,sep='\n')
print('整个向量',reg._theta,reg._theta.shape,sep='\n') 

截距
34.117399723204585
()
除了截距的
[-1.20354261e-01  3.64423279e-02 -3.61493155e-02  5.12978140e-02
 -1.15775825e+01  3.42740062e+00 -2.32311760e-02 -1.19487594e+00
  2.60101728e-01 -1.40219119e-02 -8.35430488e-01  7.80472852e-03
 -3.80923751e-01]
(13,)
整个向量
[ 3.41173997e+01 -1.20354261e-01  3.64423279e-02 -3.61493155e-02
  5.12978140e-02 -1.15775825e+01  3.42740062e+00 -2.32311760e-02
 -1.19487594e+00  2.60101728e-01 -1.40219119e-02 -8.35430488e-01
  7.80472852e-03 -3.80923751e-01]
(14,)


In [16]:
reg.predict(x_test)
reg.score(x_test,y_test)   

0.8129794056212895

### 使用多个特征 对于预测准确度 是有好处的

# sklearn的实现

In [17]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split


In [26]:
x_train1,x_test1 ,y_train1,y_test1=train_test_split(x,y,test_size=0.2,random_state=666)
print(x_train1.shape)
print(y_train1.shape)

print(x_test1.shape)
print(y_test1.shape)

(392, 13)
(392,)
(98, 13)
(98,)


In [27]:
lar=LinearRegression()
lar.fit(x_train1,y_train1)
lar.predict(x_test) 
lar.score(x_test,y_test)  #  最后也0.81多


0.8129794056212809

In [29]:
print(lar.coef_.shape)
print(lar.intercept_)

(13,)
34.117399723229845


## KNN Regressor

In [41]:
from sklearn.neighbors import KNeighborsRegressor
knn=KNeighborsRegressor()
knn.fit(x_train1,y_train1)
knn.score(x_test1,y_test1)   #  评价标准 是差的

0.5865412198300899

### 上面的KNN回归 泛化有点差，进行网格搜索优化

In [66]:
from sklearn.model_selection import GridSearchCV
param_grid=[
    {
        'weights':['uniform'],
        'n_neighbors':[i for i in range(1,11)]
    },
    {
        'weights':['distance'],     #distance 
        'n_neighbors':[i for i in range(1,11)],
        'p':[i for i in range(1,6)]
    }    
]
knn1=KNeighborsRegressor()
grid_search=GridSearchCV(knn1,param_grid,n_jobs=-1,verbose=1)
grid_search.fit(x_train1,y_train1)

D:\python\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 60 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:    1.9s finished
D:\python\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30,
                                           metric='minkowski',
                                           metric_params=None, n_jobs=None,
                                           n_neighbors=5, p=2,
                                           weights='uniform'),
             iid='warn', n_jobs=-1,
             param_grid=[{'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                          'weights': ['uniform']},
                         {'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                          'p': [1, 2, 3, 4, 5], 'weights': ['distance']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=1)

In [67]:
grid_search.best_params_
grid_search.best_score_   #与前面的衡量标准 不一样的,是 CV中的计算方法 

0.6340477954176972

In [70]:
grid_search.best_estimator_

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=1,
                    weights='distance')

In [72]:
grid_search.best_estimator_.score(x_test1,y_test1)   #与之前的衡量便准 是一样的，

0.7044357727037996